In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import math

def log_transform(data, base=math.e):
    """
    Apply logarithmic transformation to data.
    
    Parameters:
    data (array-like): Input data
    base (float): Base of logarithm (default: e - natural log)
    
    Returns:
    array: Log-transformed data
    """
    # Handle zero and negative values
    min_val = np.min(data)
    if min_val <= 0:
        offset = abs(min_val) + 1
        data_adjusted = data + offset
    else:
        data_adjusted = data
        
    if base == math.e:
        return np.log(data_adjusted)
    elif base == 10:
        return np.log10(data_adjusted)
    elif base == 2:
        return np.log2(data_adjusted)
    else:
        return np.log(data_adjusted) / np.log(base)

def sqrt_transform(data):
    """
    Apply square root transformation to data.
    
    Parameters:
    data (array-like): Input data
    
    Returns:
    array: Square root-transformed data
    """
    # Handle negative values
    min_val = np.min(data)
    if min_val < 0:
        offset = abs(min_val) + 1e-6
        data_adjusted = data + offset
    else:
        data_adjusted = data
    
    return np.sqrt(data_adjusted)

def cube_root_transform(data):
    """
    Apply cube root transformation to data.
    
    Parameters:
    data (array-like): Input data
    
    Returns:
    array: Cube root-transformed data
    """
    return np.cbrt(data)

def box_cox_transform(data, lmbda=None):
    """
    Apply Box-Cox transformation to data.
    
    Parameters:
    data (array-like): Input data (must be positive)
    lmbda (float, optional): Parameter for the Box-Cox transformation
                            If None, the function will find the optimal lambda value
    
    Returns:
    tuple: (transformed data, lambda value used)
    """
    # Handle non-positive values
    min_val = np.min(data)
    if min_val <= 0:
        offset = abs(min_val) + 1
        data_adjusted = data + offset
    else:
        data_adjusted = data
    
    if lmbda is None:
        transformed_data, lambda_value = stats.boxcox(data_adjusted)
        return transformed_data, lambda_value
    else:
        transformed_data = stats.boxcox(data_adjusted, lmbda=lmbda)
        return transformed_data, lmbda

def yeo_johnson_transform(data, lmbda=None):
    """
    Apply Yeo-Johnson transformation to data.
    
    Parameters:
    data (array-like): Input data (can be positive or negative)
    lmbda (float, optional): Parameter for the Yeo-Johnson transformation
                            If None, the function will find the optimal lambda value
    
    Returns:
    tuple: (transformed data, lambda value used)
    """
    if lmbda is None:
        transformed_data, lambda_value = stats.yeojohnson(data)
        return transformed_data, lambda_value
    else:
        transformed_data = stats.yeojohnson(data, lmbda=lmbda)
        return transformed_data, lmbda

def inverse_transform(data):
    """
    Apply inverse (reciprocal) transformation to data.
    
    Parameters:
    data (array-like): Input data (should not contain zeros)
    
    Returns:
    array: Reciprocal-transformed data
    """
    # Handle zeros
    epsilon = 1e-10
    data_adjusted = np.where(np.abs(data) < epsilon, np.sign(data) * epsilon, data)
    
    return 1.0 / data_adjusted

def power_transform(data, power):
    """
    Apply power transformation to data.
    
    Parameters:
    data (array-like): Input data
    power (float): Power to raise data to
    
    Returns:
    array: Power-transformed data
    """
    # Handle negative values for even powers
    if power % 2 == 0 and np.min(data) < 0:
        offset = abs(np.min(data)) + 1e-6
        data_adjusted = data + offset
    else:
        data_adjusted = data
    
    return np.power(data_adjusted, power)

def arcsin_transform(data):
    """
    Apply arcsin transformation to data (useful for proportions/percentages).
    
    Parameters:
    data (array-like): Input data (values should be between 0 and 1)
    
    Returns:
    array: Arcsin-transformed data
    """
    # Clip data to [0, 1] range
    data_clipped = np.clip(data, 0, 1)
    
    return np.arcsin(np.sqrt(data_clipped))

def z_score_transform(data):
    """
    Standardize data to have mean=0 and std=1 (Z-score normalization).
    
    Parameters:
    data (array-like): Input data
    
    Returns:
    array: Z-score transformed data
    """
    return stats.zscore(data)

def min_max_transform(data, feature_range=(0, 1)):
    """
    Scale data to a specified range, default [0, 1] (Min-Max scaling).
    
    Parameters:
    data (array-like): Input data
    feature_range (tuple): Desired range for transformed data
    
    Returns:
    array: Min-Max scaled data
    """
    min_val, max_val = feature_range
    data_min = np.min(data)
    data_max = np.max(data)
    
    # Handle case where all values are the same
    if data_max == data_min:
        return np.full_like(data, (min_val + max_val) / 2)
    
    return min_val + ((data - data_min) * (max_val - min_val)) / (data_max - data_min)

def robust_scale_transform(data):
    """
    Scale data using statistics that are robust to outliers (median and IQR).
    
    Parameters:
    data (array-like): Input data
    
    Returns:
    array: Robustly scaled data
    """
    median = np.median(data)
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)
    iqr = q3 - q1
    
    # Handle zero IQR
    if iqr == 0:
        iqr = 1e-10
    
    return (data - median) / iqr

def analyze_and_plot_transformations(data, transformations=None):
    """
    Analyze and plot the original data and various transformations.
    
    Parameters:
    data (array-like): Input data
    transformations (list): List of transformation functions to apply
    
    Returns:
    dict: Dictionary containing transformed data
    """
    if transformations is None:
        transformations = [
            ('Original', lambda x: x),
            ('Log', lambda x: log_transform(x)),
            ('Square Root', lambda x: sqrt_transform(x)),
            ('Cube Root', lambda x: cube_root_transform(x)),
            ('Box-Cox', lambda x: box_cox_transform(x)[0]),
            ('Yeo-Johnson', lambda x: yeo_johnson_transform(x)[0]),
            ('Z-Score', lambda x: z_score_transform(x)),
            ('Min-Max', lambda x: min_max_transform(x))
        ]
    
    results = {}
    
    n_transformations = len(transformations)
    n_cols = 3
    n_rows = (n_transformations + n_cols - 1) // n_cols
    
    plt.figure(figsize=(15, 4 * n_rows))
    
    for i, (name, transform_func) in enumerate(transformations):
        try:
            transformed = transform_func(data)
            results[name] = transformed
            
            plt.subplot(n_rows, n_cols, i + 1)
            
            # Histogram
            plt.hist(transformed, bins=30, alpha=0.7, color='skyblue')
            
            # Add a density curve
            if len(transformed) > 1:  # Need at least 2 points for kde
                try:
                    density = stats.gaussian_kde(transformed)
                    x_vals = np.linspace(min(transformed), max(transformed), 1000)
                    plt.plot(x_vals, density(x_vals) * len(transformed) * (max(transformed) - min(transformed)) / 30, 
                             color='navy', linewidth=2)
                except:
                    pass  # Skip density estimation if it fails
            
            plt.title(f'{name} Transform')
            plt.xlabel('Value')
            plt.ylabel('Frequency')
            
            # Add skewness and kurtosis info
            skew = stats.skew(transformed)
            kurt = stats.kurtosis(transformed)
            plt.annotate(f'Skewness: {skew:.4f}\nKurtosis: {kurt:.4f}', 
                         xy=(0.05, 0.95), xycoords='axes fraction',
                         fontsize=10, va='top', bbox=dict(boxstyle='round', fc='white', alpha=0.7))
            
        except Exception as e:
            plt.subplot(n_rows, n_cols, i + 1)
            plt.text(0.5, 0.5, f"Error: {str(e)}", ha='center', va='center')
            plt.title(f'{name} Transform (Failed)')
    
    plt.tight_layout()
    return results, plt.gcf()